In [1]:
%env eval_only=True
%env exp_id=maml

env: eval_only=True
env: exp_id=maml


In [2]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [3]:
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1


# xlm_cluster1_enfr

In [4]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/multi_xlm_enfr/processed
%env epoch_size=23118
%env lgs=en-fr-Bafia-Bulu-MKPAMAN_AMVOE_Ewondo

%env batch_size=32
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster1

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/multi_xlm_enfr/processed
env: epoch_size=23118
env: lgs=en-fr-Bafia-Bulu-MKPAMAN_AMVOE_Ewondo
env: batch_size=32
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster1


## MLM+TLM

In [15]:
# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

%env mlm_steps=en,fr,Bafia,Bulu,MKPAMAN_AMVOE_Ewondo,Bafia-en,Bulu-en,MKPAMAN_AMVOE_Ewondo-en,Bafia-fr,Bulu-fr,MKPAMAN_AMVOE_Ewondo-fr

! python train.py --eval_only $eval_only --exp_name mlm_tlm_cluster1enfr --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false
env: mlm_steps=en,fr,Bafia,Bulu,MKPAMAN_AMVOE_Ewondo,Bafia-en,Bulu-en,MKPAMAN_AMVOE_Ewondo-en,Bafia-fr,Bulu-fr,MKPAMAN_AMVOE_Ewondo-fr
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/29/20 01:07:15 - 0:00:00 - ============ Initialized logger ============
INFO - 05/29/20 01:07:15 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 32
      

## SupMT en cluster1

In [17]:
%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

%env stopping_criterion=valid_en-Bulu_mt_bleu,10
%env validation_metrics=valid_en-Bulu_mt_bleu
%env reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_cluster1enfr/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_cluster1enfr/maml/best-valid_mlm_ppl.pth

%env ae_steps=en,Bafia,Bulu,MKPAMAN_AMVOE_Ewondo
%env bt_steps=Bafia-en-Bafia,en-Bafia-en,Bulu-en-Bulu,en-Bulu-en,MKPAMAN_AMVOE_Ewondo-en-MKPAMAN_AMVOE_Ewondo,en-MKPAMAN_AMVOE_Ewondo-en

%env mt_steps=Bafia-en,en-Bafia,Bulu-en,en-Bulu,MKPAMAN_AMVOE_Ewondo-en,en-MKPAMAN_AMVOE_Ewondo
! python train.py --eval_only $eval_only --exp_name SupMT_cluster1en --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: eval_bleu=true
env: stopping_criterion=valid_en-Bulu_mt_bleu,10
env: validation_metrics=valid_en-Bulu_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_cluster1enfr/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_cluster1enfr/maml/best-valid_mlm_ppl.pth
env: ae_steps=en,Bafia,Bulu,MKPAMAN_AMVOE_Ewondo
env: bt_steps=Bafia-en-Bafia,en-Bafia-en,Bulu-en-Bulu,en-Bulu-en,MKPAMAN_AMVOE_Ewondo-en-MKPAMAN_AMVOE_Ewondo,en-MKPAMAN_AMVOE_Ewondo-en
env: mt_steps=Bafia-en,en-Bafia,Bulu-en,en-Bulu,MKPAMAN_AMVOE_Ewondo-en,en-MKPAMAN_AMVOE_Ewondo
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/29/20